In [274]:
import numpy as np
import json
from edward.models import Poisson
from edward.models import Gamma as IGR
import tensorflow as tf
import gc

In [275]:
with open('package_id_dict.json', 'r') as f:
    package_id_dict = json.load(f)

In [276]:
id_package_dict = {x: n for n, x in package_id_dict.items()}

In [277]:
with open('manifest_id_dict.json', 'r') as f:
    manifest_id_dict = json.load(f)

In [278]:
manifest_id_dict = {n: set(x) for n, x in manifest_id_dict.items()}

In [279]:
input_list = ['io.vertx:vertx-core', 'io.vertx:vertx-web']

In [280]:
input_id_set = set()
unknown_list = []
for package_name in input_list:
    p_id = package_id_dict.get(package_name, -1)
    if p_id == -1:
        unknown_list.append(package_name)
    else:
        input_id_set.add(p_id)

In [281]:
input_id_set

{1729, 1861}

In [282]:
for n, x in manifest_id_dict.items():
    if x == input_id_set:
        break
else:
    n = -1

In [283]:
sess = tf.InteractiveSession()
init = tf.global_variables_initializer()
init.run()

In [284]:
n = int(n)
n

571

In [285]:
rating = np.load("rating_result_matrix.npy")
beta = np.load("item_result_matrix.npy")

In [286]:
theta = []
if n == -1:
    a = a_c = c = c_c = 0.5
    b_c = d_c = 0.99
    K = 10
    epsilon = IGR(a_c, a_c/b_c).eval()
    theta_func = IGR(a, epsilon).eval()
    theta.append(epsilon*theta_func)
    theta = np.array(theta*K)
    result = Poisson(np.dot(theta, np.transpose(beta))).eval()
    maxn = result.max()
    min_max = maxn-rating.min()
    for i in range(12537):
        result[i] = (maxn-result[i])/min_max
else:
    result = rating[n]

In [287]:
m = rating.mean()
m

0.12138094119577054

In [288]:
result.mean()

0.11944124423963132

In [289]:
reco = list(result.nonzero()[0])

In [290]:
from collections import defaultdict
companion = {}
# c_mean=0
for p_id in reco:
    prob_score = result[p_id]
    if prob_score > result.mean():
        package_name = id_package_dict[p_id]
        companion[package_name] = prob_score
        c_mean += prob_score

In [291]:
sorted(companion.items(), key=lambda x: x[1], reverse=True)[:10]

[('io.springfox:springfox-swagger-ui', 0.5268817204301075),
 ('org.springframework.boot:spring-boot-starter-redis', 0.5268817204301075),
 ('io.jsonwebtoken:jjwt', 0.5053763440860215),
 ('org.apache.tomcat.embed:tomcat-embed-jasper', 0.3763440860215054),
 ('javax.servlet:jstl', 0.3225806451612903),
 ('com.alibaba:druid', 0.3010752688172043),
 ('org.springframework.boot:spring-boot-starter-data-mongodb',
  0.27956989247311825),
 ('com.shopizer:sm-search', 0.26881720430107525),
 ('org.springframework.cloud:spring-cloud-starter-eureka',
  0.26881720430107525),
 ('org.springframework.cloud:spring-cloud-starter-feign', 0.26881720430107525)]